<br>
<p style="font-size: 35px;
          text-align:center;
          font-family: 'Times New Roman';
          font-variant: small-caps;
          font-weight: bold;
          line-height: 50px;"> Propuesta Solución 2 Detección de Suciedad en Paneles Solares</p>
<br>

Con el fin de detectar ambos tipos de suciedad en los panel (cafe y blanca) se realiza una serie de transformaciones que logra resaltar ambos tipos:

* Se cambia al espacio de colo CMY
* Se aplica una tranformacion gamma con parámetros a = 1.2 y a = 0.4
* luego se binariza ambas imagenes con un threshold de 240
* Se aplica una transformación morfológica de apertura para eliminar las lineas del panel que no correponden a la suciedad 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import *
import cv2
import sys
sys.path.append('./utils/')
from utils.GUI_Color import *
from utils.funciones_utiles import *
from pathlib import Path

# Se crea la ruta cwd para encontrar la ubicación del archivo
cwd = Path.cwd()

# Se encuentra la carpeta donde se encuentran las imágenes
DATA_PATH = cwd / 'datasets/Suciedad_tipo1'

# Se encuentra una lista con la ubicación de las imagenes
images = list(DATA_PATH.glob('*.jpg'))

#Se encuentra la ubicación relativa de las imágenes
images =[image.relative_to(cwd) for image in images]

# Se encuentra la carpeta donde se encuentran las imágenes
DATA_PATH = cwd / 'datasets/Suciedad_tipo2'

# Se encuentra una lista con la ubicación de las imagenes
images = list(DATA_PATH.glob('*.jpg'))

#Se encuentra la ubicación relativa de las imágenes
images_2 =[image.relative_to(cwd) for image in images]

def f(i):
    
    img_rgb_1=cv2.imread(str(images[i]))
    img_rgb_2=cv2.imread(str(images_2[i]))
    # Se cambia a un espacio de color RGB

    img_rgb_1=cv2.cvtColor(img_rgb_1, cv2.COLOR_BGR2RGB)
    img_rgb_2=cv2.cvtColor(img_rgb_2, cv2.COLOR_BGR2RGB)
    
       
    img_C_CMY_1 = 255 - img_rgb_1[:,:,0]
    img_M_CMY_1 = 255 - img_rgb_1[:,:,1]
    img_Y_CMY_1 = 255 - img_rgb_1[:,:,2]

    img_C_CMY_2 = 255 - img_rgb_2[:,:,0]
    img_M_CMY_2 = 255 - img_rgb_2[:,:,1]
    img_Y_CMY_2 = 255 - img_rgb_2[:,:,2]

    res_1 = gamma_correction(img_C_CMY_1, 1.2, 0.4)
    res_1 = histogram_expansion(res_1)

    res_2 = gamma_correction(img_C_CMY_2, 1.2, 0.4)
    res_2 = histogram_expansion(res_2)
  

    img_rgb = cv2.cvtColor(cv2.imread(str("datasets/Limpio.jpg")),cv2.COLOR_BGR2RGB)
    panel_B_LAB= cv2.cvtColor(img_rgb,cv2.COLOR_RGB2LAB)[:,:,2]

    # Se crea la mascara 
    panel_mask=np.zeros_like(img_rgb)
    panel_mask[(panel_B_LAB < 116)]=True

    panel_mask = panel_mask[:,:,0]
    
    elem = morph.disk(2)
    panel_mask = morph.binary_erosion(panel_mask,elem)

    img_out_1=img_rgb_1.copy()
    img_out_2=img_rgb_2.copy()

    img_out_1[(res_1<240)&(panel_mask==True)]=[20,200,0]
    img_out_2[(res_2<240)&(panel_mask==True)]=[20,200,0]
    
    panel_mask_out1=np.zeros_like(img_out_1)
    panel_mask_out2=np.zeros_like(img_out_2)
    panel_mask_out1[(res_1<240)&(panel_mask==True)]=255 
    panel_mask_out2[(res_2<240)&(panel_mask==True)]=255 

    radius = 1

    selem = morph.disk(radius)

    apertura1 = morph.binary_opening(panel_mask_out1[:,:,0],selem)

    radius = 1

    selem = morph.disk(radius)

    apertura2 = morph.binary_opening(panel_mask_out2[:,:,0],selem)

    
    fig, arreglo_plots = plt.subplots(4,2, figsize=(15, 30))
    arreglo_plots[0,0].imshow(img_rgb_1)
    arreglo_plots[0,1].imshow(img_rgb_2)
    arreglo_plots[1,0].imshow(res_1, cmap='gray')
    arreglo_plots[1,1].imshow(res_2, cmap='gray')
    arreglo_plots[2,0].imshow(img_out_1, cmap='gray')
    arreglo_plots[2,1].imshow(img_out_2, cmap='gray')
    arreglo_plots[3,0].imshow(apertura1, cmap='gray')
    arreglo_plots[3,1].imshow(apertura2, cmap='gray')

_ = interact(f, i=widgets.Play(value=0, min=0, max=20, step=1, interval=500))